In [ ]:
import pandas as pd
import os
import xml.etree.ElementTree
import sys

In [ ]:
def get_first(Node):
    for x in Node:
        return x

In [ ]:
r = xml.etree.ElementTree.parse('../../parameters/parametersM20.xml').getroot()
libNode = get_first(r.iter(tag='mocapToRobotLibPath'))
libPath = libNode.text
sys.path.append(os.path.join(libPath,'externalLibs','mine'))
import PathGraph as pg

In [ ]:
%run SubSamplePlotLib.py

In [ ]:
fileNode = get_first(r.iter(tag='subsamplePlotC3DFile'))
filePath = fileNode.text
filePath = filePath.replace('._V3D.c3d.txt','.txt')
filePathNE = os.path.splitext(filePath)[0]
plotFile =  filePathNE + '.' + r'plot.txt'
indicesFile = filePathNE + '.' + r'indices.txt'
indicesUniformFile = filePathNE + '.' + r'indicesUniform.txt'
errorsFile = filePathNE + '.' + r'ssErrors.txt'
errorsFileUni = filePathNE + '.' + r'ssErrorsUniform.txt'

In [ ]:
#read the matlab generated CSV files
points = pd.read_csv(plotFile, header=None)
proxEnd = points[[0,1,2]].copy()
distEnd = points[[3,4,5]].copy()
rotation = points[[6,7,8]].copy()

#assign them columns for plotting
proxEnd.columns = ['x','y','z']
distEnd.columns = ['x','y','z']
rotation.columns = ['x','y','z']

#create dataframes that contain only the kept variables
keptIndices = pd.read_csv(indicesFile,header=None)
keptIndicesUniform = pd.read_csv(indicesUniformFile,header=None)

proxEndKept = proxEnd.iloc[keptIndices.values[0]-1]
distEndKept = distEnd.iloc[keptIndices.values[0]-1]
rotationKept = rotation.iloc[keptIndices.values[0]-1]

proxEndKeptUni = proxEnd.iloc[keptIndicesUniform.values[0]-1]
distEndKeptUni = distEnd.iloc[keptIndicesUniform.values[0]-1]
rotationKeptUni = rotation.iloc[keptIndicesUniform.values[0]-1]

#read the error file
error = pd.read_csv(errorsFile, header=None)
error.columns = ['p','d','o']
errorUni = pd.read_csv(errorsFileUni, header=None)
errorUni.columns = ['p','d','o']

In [ ]:
cameraDist = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=3.0, y=0.6, z=0.6))

In [ ]:
cameraProx = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=3.5, y=0.8, z=0.8))

In [ ]:
#plot 3D
pg.plotXYZC('graphs/proximal.html', cameraProx, (proxEnd, 'Proximal', 'red', 'lines'), 
            (proxEndKept, 'Proximal Subsample', 'blue', 'lines+markers'), (proxEndKeptUni, 'Proximal Subsample Uniform', 'green', 'lines+markers'))
#pg.plotXYZC('graphs/distal.html', cameraDist, (distEnd, 'Distal', 'red', 'lines'), (distEndKept, 'Distal Subsample', 'blue', 'markers'))
#pg.plotXYZC('graphs/orient.html', camera, (rotation, 'Rotation', 'Red', 'lines'), (rotationKept, 'Rotation Subsample', 'Blue', 'lines+markers'))

In [ ]:
#plot 2D
plot2DSub('graphs/2D.html', os.path.basename(filePathNE), proxEnd, proxEndKept, distEnd, distEndKept, rotation, rotationKept)

In [ ]:
#plot error
pg.plot2D1X3('graphs/error.html','Subsampling Error', (error['p']*1000, 'Proximal End', 'Index', 'Error (mm)'), (error['d']*1000, 'Distal End', 'Index', 'Error (mm)'), (error['o'], 'Orientation', 'Index', 'Error (deg)'), inline=False)
pg.plot2D1X3('graphs/errorUni.html','Subsampling Error', (errorUni['p']*1000, 'Proximal End', 'Index', 'Error (mm)'), (errorUni['d']*1000, 'Distal End', 'Index', 'Error (mm)'), (errorUni['o'], 'Orientation', 'Index', 'Error (deg)'), inline=False)